#### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import os, glob, shutil
import openpyxl
import re # For regular expressions
from LatLon23 import string2latlon
import requests

# Import own module
from PyCurator.PyCurator import PyCurator as pc
from PyCurator.PyCurator import PanData
import importlib
importlib.reload(pc)
importlib.reload(PanData)

#### Load parameter database

In [ ]:
# Download PANGAEA parameters
db = PanData.get_PanParameters("C:\\Users\\marum2022\\Documents\\MARUM\\Tickets\\Database\\")

#### Setup curation folder

Set file path

In [ ]:
# Enter author folder
author = "Spatafora_etal_PDI-33204"
# Store file path
file_path = f'C:\\Users\\marum2022\\Documents\\MARUM\Tickets\\Open tickets\\{author}\\'
# Change the current working directory
os.chdir(file_path)
os.getcwd()

Create working folders and move data files

In [ ]:
# Create curation folders
new_folders = ["Dataset", "Dataset_curated", "Parameters", "Events", "Plots"]

for folder in new_folders:
    # Check if folder already exists
    if os.path.isdir(folder):
        print(f'{folder} already exists')
        continue
    # if not create it       
    else:
        os.mkdir(folder)

# And move all data files into Dataset folder
[shutil.copy(file, f'{file_path}Dataset\\') for file in pc.list_files(file_path)]
# Remove files
[os.remove(file) for file in pc.list_files(file_path)]

Read user files

In [ ]:
# Read user files
df_dict = pc.load_df(sub_folder= file_path + "Dataset\\")

#### General information about dataframes

General information for all data files

In [ ]:
# Save dataframe names
df_names = df_dict.keys()
print(df_names)
# General information for all data files
for df_name, df in df_dict.items():
    print("-"*70)
    print(f'The dataset {df_name} has {df.shape[0]} rows and {df.shape[1]} columns')
    print(pd.concat([df.head(2), df.tail(2)]).T)

# Individual file processing

In [ ]:
# Create empty dictionary to store curated data frames
df_curated = {}

## Dataset 1: Dataset

#### How does the dataset look like?

In [ ]:
# Print all file names
df_names

In [ ]:
# Define dataset name
df_name = 'Dataset'
# Create a copy of dataframe
df = df_dict[df_name].copy()
print(f'The dataframe {df_name} has {df.shape[0]} rows and {df.shape[1]} columns')
pd.concat([df.head(3), df.tail(3)])

Remove empty columns

In [ ]:
# First delete any empty columns
df.dropna(how='all', axis=1, inplace=True)

In [ ]:
# General overview and data types
df.info()

#### Replace user with database parameters

In [ ]:
# Print column labels line by line with added [#]
for x in df.columns:
    print(f"'{x} []',")

Find close matches

In [ ]:
# # Search for close matches in PANGAEA database for single string
# search_term = "Distance from shelter []"
# PanData.get_close_match([search_term], db["Parameter"] + " [" + db["Unit"] + "]", n_matches = 15).T

Find start search term

In [ ]:
# # Find all terms starting with search term
# search_term = "Distance from shelter"
# str_start = db["Parameter"][db["Parameter"].str.lower().str.contains(search_term.lower(), )]
# str_start
# # Subfilter results with further match
# str_start[str_start.str.contains("")]

New parameters

In [ ]:
# New events to be replaced
new_params = [
            'Class',
            'Order',
            'Family',
            'Species',
            'Species, common name',
            'IUCN status',
            'Life stage',
            'DATE/TIME',
            'Month',
            'Year',
            'Location',
            'LATITUDE',
            'LONGITUDE',
            'Photographer',
            'Comment'
            ]
# Use simple replace method
df.columns = new_params
df.head(2)

#### Conversions

Delete all leading and trailing white space

In [ ]:
# Remove all leading and trailing white spaces 
cols = df.select_dtypes(['object']).columns

# Remove all leading and trailing white spaces 
for col in cols:
    # Remove leading and trailing white space
    df[col].replace(r"^ +| +$", r"", regex=True, inplace=True)
    # Remove double white spaces
    df[col].replace(r"\s+", r" ", regex=True, inplace=True)

#### Convert to PANGAEA standard date format

In [ ]:
# Convert to PANGAEA date format
df["DATE/TIME []"] = pc.toPangaeaDate(df["DATE/TIME []"], '%Y-%m-%d')

Spell out abbreviations

In [ ]:
df["abbreviation"].unique()

In [ ]:
# Create list with abbreviations
short = ("NE",
              "DD",
              "LC",
)

# Create list with full names
long = ("Not Evaluated", 
              "Data Deficient", 
              "Least Concern", 
)

# Create dictionary from long and short names
labels = dict(zip(short, long))

# Replace the values in 'Name' column with the dictionary
df = df.replace({"abbreviation": labels})

df["abbreviation"].unique()

Check for other strings in numeric values

In [ ]:
pc.count_char(df, "[a-zA-Z]")

Convert all commas to dots

In [ ]:
df = pc.replace_char(df)

Convert object to numeric

In [ ]:
# Convert multiple columns of dataframe to numeric
df.iloc[:, 2:] = df.iloc[:, 2:].apply(pd.to_numeric)
#Check numeric features for text entries
df.info()

#### Add comments

In [ ]:
# Add methods details for species only
df.columns.values[2:-3] = df.columns.values[2:-3] + "//*counted from 10 samples"

#### Add event label column

In [ ]:
# Insert event column at first position
df.insert(0,'Event []', "EN18224-4")

#### Find matching parameters in database

Find unmatched parameters in database and create import table

In [ ]:
# Find matched and unmatched items of one dataframe in another
_, unmatched = PanData.find_df_matches(PanData.create_find_df(PanData.strip_comments(df.columns)), db)
unmatched

In [ ]:
# close_matches = PanData.close_db_matches(unmatched, db, n_matches=5)
# close_matches

Find unmatched parameters in database

In [ ]:
# Create new data table without comments in header
df2 = df.copy()
df2.columns = PanData.strip_comments(df2.columns)

# Get unmatched dataframe entries
unmatched_param_df = PanData.get_unmatched_df(df2, db)
unmatched_param_df.head(3)

Create import table

In [ ]:
# Find unmatched parameters in database and create import table
imp_table = PanData.get_imp_param(unmatched_param_df,
                        file_path = f'{file_path}Parameters\\',
                        DefaultMethodID = "", 
                        ReferenceID = "", 
                        Description = "", 
                        url_parameter = "",
                        df_name = df_name,
                        dec_limit = 4)
imp_table

#### Store curated dataframe in dictionary

In [ ]:
df_curated[df_name] = df
print(f'Dataset: {df_name}')
df_curated[df_name].head(3)

#### Assign unique species names

#### Save unique species names for taxon matching

In [ ]:
df["Species []"].unique()

In [ ]:
# Create new species folder
folder = "Species_naming"
# Check if folder already exsits
if not os.path.isdir(folder):
    os.mkdir(folder)
# Save species names
species = df["Stomach content []//*found in Gadus morhua//$11775"].drop_duplicates().to_frame()
# Rename header
species.columns = ["name"]
# Save as csv file
species
species.to_csv(f'{file_path}{folder}\\{df_name}_Species.txt', index=False, sep="\t", encoding='ansi')

#### Load ITIS match list
Saved species list will be matched in the taxon match tools of:
- [WoRMS](https://www.marinespecies.org/aphia.php?p=match) 
- [ITIS](https://www.itis.gov/taxmatch.html)

In [ ]:
# Load matched taxon file
itis = pd.read_csv(f'{file_path}{folder}\\{df_name}_Species_ITIS.csv', sep="|", encoding = "ISO-8859-1")
# Convert TSN to integer
itis.TSN = itis.TSN.astype('Int64')
itis.head(2)

Match TSN to dataframe

In [ ]:
# Remove duplicates from itis dataframe
itis = itis.drop_duplicates(subset="Scientific Name")
# Merge two dataframes on two common columns
df = pd.merge(df, itis.iloc[:,[0, 1]],
    how="left",
    left_on="Species []",
    right_on= "Scientific Name"
).drop("Scientific Name", axis='columns')

Create Species (Semantic URI) columns from  TSN

In [ ]:
# Create semantic URI by pasting urn and TSN together
df.insert(4, "Species, unique identification []", [f'urn:lsid:itis.gov:itis_tsn:{str(x)}' for x in df.TSN])
# Remove TSN column
df = df.drop(["TSN"], axis = 1)

### Create new events based on unique latitude and longitude

First generate events based on unique lat/long

In [ ]:
# First extract Location and lat and long
events = {}
for key, df in df_curated.items():
    events[key] = df.loc[:, ['Location []', '1600', '1601']]
# Then concatenate all data  
events = pd.concat(events, ignore_index = True)

# Create new events based on unique latitude and longitude
# Define target columns
cols = ['1600', '1601']
# Sort dataframe
events = events.sort_values(by=cols, ascending=False)
# Drop duplicates
events = events.drop_duplicates(subset = cols, keep = "first")
# Create event name
events.insert(0, "Event []", [f'CO2_Behav_2022_{x}' for x in range(1,len(events)+1)])
events                                    

Then merge those with data frames based on lat/lon

In [ ]:
# Merge an event column to all dataframes in a dictionary by matching with lat and long
df_new = {}
for key, df in df_curated.items():
    df_new[key] = df.loc[:, ['1600', '1601']]
    # Merge events with original dataframe
    df = pd.merge(df, events.loc[:, ['Event []', '1600', '1601']],
        how="left",
        on=['1600', '1601'],
    )
    # Move Event column to first column
    df.insert(0, 'Event []', df.pop("Event []"))
    # Store data with event column in dictionary
    df_new[key] = df
    #print(df_new[key])

## Final data check

In [ ]:
# Variable descriptive stats of multiple files
for key, df in df_curated.items():
    print("-"*70)
    print(f'The dataset {key} has {df.shape[0]} rows and {df.shape[1]} columns')
    print(f'Data types:\n{df.dtypes.value_counts()}')
    print(df.head(2).T)

#### Check integers

In [ ]:
# Variable descriptive stats of multiple files
for key,  df in df_curated.items():
    if (df.dtypes != "object").any():
        print("-"*70)
        print(f'Dataset: {key}')
        print(round(df.describe().loc[["mean", "min", "max"]], 2).T)

#### Plot all numeric features

In [ ]:
# Plot all numeric features for each dataframe
for key,  df in df_curated.items():
    # Lineplot for each numeric feature
    pc.plot_data(df, plot_name=key, file_path=f'{file_path}Plots')

#### Check objects

In [ ]:
# Summary for all non-numeric data types
for key, df in df_curated.items():
    print("-"*70)
    print(f'The dataset {key} has {df.shape[0]} rows and {df.shape[1]} columns')
    if (df.dtypes == "object").any():
        print(df.describe(exclude=[np.number]).T)

#### Join new parameter tables

In [ ]:
# Read metadata files
param_dict = pc.load_df(sub_folder= f'{file_path}Parameters\\')

# Join all dataframes in the dictionary
param_imp = pd.concat(param_dict.values(), ignore_index=True)

# Remove duplicate rows
param_imp = param_imp.drop_duplicates(subset="ParameterName", ignore_index=True)

# Save final parameter import table
param_imp.to_csv(f'{file_path}Parameters\\ParamImp_{author}_MOellermann.txt', index=False, sep="\t", encoding='utf-8')

## Save file

In [ ]:
df_curated.keys()

In [ ]:
# Save curated files
for key, df in df_curated.items():
    # Create subfolder path
    sub_folder = f'{file_path}Dataset_curated\\{key}\\'
    # Check if folder already exists
    if not os.path.isdir(sub_folder):
        # if not create it    
        os.mkdir(sub_folder)

    # Save file
    # as tab delimited csv file
    df.to_csv(f'{sub_folder}{key}_curated.txt', index=False, sep="\t", encoding='ansi')
    # as excel
    if df.shape[0] < 1048576: # Checks maximum allowed sheet size for excel
        df.to_excel(f'{sub_folder}{key}_curated.xlsx', index=False)
    print(f'Dataset {key} saved')

Save events

In [ ]:
# First extract event details
events = df[['Event []', '1600', '1601', 'Site []']]
# Save as csv
events.to_csv(f'{file_path}Events//MassImport_Event.txt', index=False, sep="\t", encoding='utf-8')